In [1]:
%pip install accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7 datasets

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 0.0/92.5 MB ? eta -:--:--
   --- ------------------------------------ 7.1/92.5 MB 33.6 MB/s eta 0:00:03
   ------ --------------------------------- 13.9/92.5 MB 33.5 MB/s eta 0:00:03
   ------- -------------------------------- 17.6/92.5 MB 27.0 MB/s eta 0:00:03
   --------- ------------------------------ 21.2/92.5 MB 24.4 MB/s eta 0:00:03
   ---------- ----------------------------- 25.4/92.5 MB 23.3 MB/s eta 0:00:03
   ------------ --------------------------- 29.4/92.5 MB 22.7 MB/s eta 0:00:03
   -------------- ------------------------- 33.8/92.5 MB 22.4 MB/s eta 0:00:03
   ---------------- ----------------------- 38.0

  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [863 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-312
      creating build\lib.win-amd64-cpython-312\tokenizers
      copying py_src\tokenizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers
      creating build\lib.win-amd64-cpython-312\tokenizers\models
      copying py_src\tokenizers\models\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\models
      creating build\lib.win-amd64-cpython-312\tokenizers\decoders
      copying py_src\tokenizers\decoders\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\decoders
      creating build\lib.win-amd64-cpython-312\tokenizers\normalizers
      copying py_src\tokenizers\normalizers\__init__.py -> build\lib.win-amd64-cpython-312\tokenizers\normalizers
      creating build\lib

In [4]:
import os
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer
import pandas as pd

In [6]:
base_model = "NousResearch/Llama-2-7b-hf"
data = pd.read_csv('../data/sciq_train.csv')
data['text'] = data.apply(lambda row: f"Question: {row['question']} Answer: {row['correct_answer']} Reason: {row['support']}", axis=1)
data = data.iloc[0:1000]
dataset = Dataset.from_pandas(data)
new_model = "llama-2-7b-sciq"

In [9]:
compute_dtype = getattr(torch, "float16")
quant_config = BitsAndBytesConfig(load_in_4bit=True,
                                  bnb_4bit_quant_type="nf4",
                                  bnb_4bit_compute_dtype=compute_dtype,
                                  bnb_4bit_use_double_quant=False)

In [11]:
model = AutoModelForCausalLM.from_pretrained(base_model, quantization_config=quant_config, device_map={"": 0})
model.config.use_cache = False
model.config.pretraining_tp = 1

ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_params = LoraConfig(lora_alpha=16, lora_dropout=0.1, r=64, bias="none", task_type="CAUSAL_LM")

In [ ]:
training_params = TrainingArguments(output_dir="./results",
                                    num_train_epochs=1,
                                    per_device_train_batch_size=4,
                                    gradient_accumulation_steps=1,
                                    optim="paged_adamw_32bit",
                                    save_steps=25,
                                    logging_steps=25,
                                    learning_rate=2e-4,
                                    weight_decay=0.001,
                                    fp16=False,
                                    bf16=False,
                                    max_grad_norm=0.3,
                                    max_steps=-1,
                                    warmup_ratio=0.03,
                                    group_by_length=True,
                                    lr_scheduler_type="constant",
                                    report_to="tensorboard")

In [ ]:
trainer = SFTTrainer(model=model,
                     train_dataset=dataset,
                     peft_config=peft_params,
                     dataset_text_field="text",
                     max_seq_length=None,
                     tokenizer=tokenizer,
                     args=training_params,
                     packing=False)

In [ ]:
trainer = SFTTrainer(model=model,
                     train_dataset=dataset,
                     peft_config=peft_params,
                     dataset_text_field="text",
                     max_seq_length=None,
                     tokenizer=tokenizer,
                     args=training_params,
                     packing=False)
trainer.train()